In [4]:
import numpy as np

In [5]:
import logging
import os
import pandas as pd
from common import indicators
import sys
from datetime import datetime, timedelta
from common.get_api_data import *

logging.basicConfig(level=logging.INFO)
import common.application as ap
import pytz
from sklearn.preprocessing import MinMaxScaler

In [6]:
token='408065' # infy
token='738561' # infy
token = '3676417' # Apolo pipes
token="738561" # reliance

In [7]:
import pytz
tz = pytz.timezone('Asia/Kolkata')
def getDateTime():
    datetime_obj_hour_fwd = datetime.now(tz)
    return datetime_obj_hour_fwd


datetime_obj_hour_fwd = getDateTime() + timedelta(hours=1)
from_date = str(datetime_obj_hour_fwd - timedelta(days=14)).split(" ")[0]
to_date = str(datetime_obj_hour_fwd).split(" ")[0]
historical_data=""
interval="10minute"

In [ ]:
historical_data=get_data(token,from_date,to_date,interval,historical_data,counter=0)
token

In [ ]:
df=pd.DataFrame(historical_data)
window = 20
no_of_std = 2
rolling_mean = df['close'].rolling(window).mean()
rolling_std = df['close'].rolling(window).std()

df['Bollinger High'] = rolling_mean + (rolling_std * no_of_std)
df['Rolling Mean'] = rolling_mean
df['Bollinger Low'] = rolling_mean - (rolling_std * no_of_std)

# Graph Plot

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(16,8))
plt.plot(df[258-50:].date,df[258-50:]['Rolling Mean'],df[258-50:].date,df[258-50:]['Bollinger High'],df[258-50:].date,df[258-50:]['Bollinger Low'])
plt.ylabel('some numbers')
plt.show()

# Scale the Data

In [ ]:
l=[]
for i in range(0,df.shape[0]):
    l.append(i)
    
scaler = MinMaxScaler()

In [ ]:

df['diff_a']=(df['Bollinger High']-df['Bollinger Low']).round(2)
L=pd.DataFrame(np.around(scaler.fit_transform(df.filter(['diff_a']).values), decimals=2),columns=['DIFF'])
L['key']=l
B=pd.DataFrame(np.around(scaler.fit_transform(df.filter(['Bollinger Low']).values), decimals=2),columns=['LB'])
B['key']=l
H=pd.DataFrame(np.around(scaler.fit_transform(df.filter(['Bollinger High']).values), decimals=2),columns=['LH'])
H['key']=l
k=B.merge(H,on='key')
k=k.merge(L,on='key')
df['key']=l
df_u=df.merge(k,on=['key'],how="inner")
df_u['diff']=(df_u.LH-df_u.LB)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(16,8))
df_k=df_u[258-50:]
plt.plot(df[258-50:].date,df_k['Rolling Mean'],df_k.date,df_k['Bollinger High'],df_k.date,df_k['Bollinger Low'],df_k.date,df_k['LB'])
plt.ylabel('some numbers')
plt.show()


In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(16,4))
df_k=df_u[258-110:]
plt.plot(df_k.date,df_k['LB'])
plt.ylabel('some numbers')
plt.show()


In [ ]:
df_u['diff']=(df_u.LH-df_u.LB)
df_u['diff_a']=(df_u['Bollinger High']-df_u['Bollinger Low'])

In [ ]:
D=0
df_u[265-D:]
L=pd.series(np.around(scaler.fit_transform(df_u[265-D:].filter(['diff_a']).values), decimals=1),columns=['DIFF_S'])
L['key']=l[265-D:]
df_u[265-D:].merge(L,on='key').to_csv("report.csv")

In [ ]:

df_u.round(0).to_csv("screening.csv",index=False)

In [ ]:
df_u[:267].diff_a.round(0)

In [ ]:
from scipy import interpolate
import matplotlib.pyplot as plt
import numpy as np

x = np.linspace(-15,15,100)

In [ ]:
y = np.sin(x)

In [ ]:
y

In [ ]:
tck = interpolate.splrep(x,y)

In [ ]:
tck

In [ ]:
plt.plot(x,y)

In [ ]:
x=[5,10,15,20]

In [ ]:
y=[47,45,44,5]

In [ ]:
plt.plot(x,y)
tck = interpolate.splrep(x,y)
x0 = 15
y0= 44
y0 = interpolate.splev(x0,tck)
dydx = interpolate.splev(x0,tck,der=1)
plt.plot(x,tngnt(x), label="tangent")
tngnt = lambda x: dydx*x + (y0-dydx*x0)

In [ ]:
dydx

In [ ]:
plt.plot(x,y)
plt.plot(x0,y0, "or")


plt.legend()
plt.show()

In [ ]:
tngnt(x)

In [ ]:
plt.plot(x,tngnt(x), label="tangent")

In [ ]:
y=np.array(df_u['Rolling Mean'][240:265])

In [ ]:
counter=0
x_list=[]
for i in y:
    x_list.append(counter*10)
    counter=counter+1
x=np.array(x_list)

In [ ]:
x

In [ ]:
plt.plot(x,y)
tck = interpolate.splrep(x,y)
c=23
x0 = x[c]
y0= y[c]
y0 = interpolate.splev(x0,tck)
dydx = interpolate.splev(x0,tck,der=1)
plt.plot(x,tngnt(x), label="tangent")
tngnt = lambda x: dydx*x + (y0-dydx*x0)

In [ ]:

def is_swing(df):
    temp_df = df.iloc[-3:]
    if holding == 'up':
        high_values = list(temp_df['high'])
        if high_values[2] < high_values[0] < high_values[1]:
            return True
    else:
        low_values = list(temp_df['low'])
        if low_values[1] < low_values[0] < low_values[2]:
            return True

In [ ]:
from common import indicators

In [ ]:
global historical_data, rsi, rsi_slope, last_close, swing,last_high,last_low,sma,ub,mb,lb,band_diff_trend,current_price
swing = False
current_price = get_price(token, 0.0)
historical_data = get_data(token, from_date, to_date, "10minute", historical_data)
df = pd.DataFrame(historical_data)
df = indicators.RSI(df,period=14)
df=indicators.SMA(df, "close", "sma_7", 7)
df['RSI_Trend'] = df['RSI_14'].diff()
window = 20
no_of_std = 2
rolling_mean = df['close'].rolling(window).mean()
rolling_std = df['close'].rolling(window).std()
df['ub'] = rolling_mean + (rolling_std * no_of_std)
df['mb'] = rolling_mean
df['lb'] = rolling_mean - (rolling_std * no_of_std)
df['band_diff']=(df['ub']-df['lb']).round(2)
df['band_diff_trend'] = df['band_diff'].diff().round(2)
df['band_diff_trend_fited']=pd.Series(np.around(scaler.fit_transform(df.filter(['band_diff_trend'])), decimals=2).reshape(df.shape[0])*10)
tail_dict = df.tail(1).to_dict('list')

sma = tail_dict['sma_7'][0]
rsi = tail_dict['RSI_14'][0]
rsi_slope = tail_dict['RSI_Trend'][0]
last_close = tail_dict['close'][0]
last_low = tail_dict['low'][0]
last_high = tail_dict['high'][0]
ub=tail_dict['ub'][0]
mb=tail_dict['mb'][0]
lb=tail_dict['lb'][0]


df['trend_diff']=10*(pd.Series(np.around(scaler.fit_transform(df.filter(['ub'])), decimals=2).reshape(df.shape[0]))-pd.Series(np.around(scaler.fit_transform(df.filter(['lb'])), decimals=2).reshape(df.shape[0])))
band_diff_trend=tail_dict['band_diff_trend'][0]


In [ ]:
global historical_data, rsi, rsi_slope, last_close, swing,last_high,last_low,sma,ub,mb,lb,band_diff_trend,current_price
swing = False
current_price = get_price(token, 0)
historical_data = get_data(token, from_date, to_date, "10minute", historical_data)
df = pd.DataFrame(historical_data)
df = indicators.RSI(df,period=14)
df=indicators.SMA(df, "close", "sma_7", 7)
df['RSI_Trend'] = df['RSI_14'].diff()
window = 20
no_of_std = 2
rolling_mean = df['close'].rolling(window).mean()
rolling_std = df['close'].rolling(window).std()
df['ub'] = rolling_mean + (rolling_std * no_of_std)
df['mb'] = rolling_mean
df['lb'] = rolling_mean - (rolling_std * no_of_std)
df['band_diff']=(df['ub']-df['lb']).round(2)
df['band_diff_trend'] = df['band_diff'].diff().round(0)
tail_dict = df.tail(1).to_dict('list')

sma = tail_dict['sma_7'][0]
rsi = tail_dict['RSI_14'][0]
rsi_slope = tail_dict['RSI_Trend'][0]
last_close = tail_dict['close'][0]
last_low = tail_dict['low'][0]
last_high = tail_dict['high'][0]
ub=tail_dict['ub'][0]
mb=tail_dict['mb'][0]
lb=tail_dict['lb'][0]
band_diff_trend=tail_dict['band_diff_trend'][0]

In [ ]:
df.to_csv("trend_reliance.csv")

In [ ]:
df

In [ ]:
np.around(scaler.fit_transform(df.filter(['band_diff_trend'])), decimals=1).reshape(df.shape[0])

In [ ]:
df.shape[0]

In [ ]:
vector_1 = [13, 1]
vector_2 = [14, 0]

unit_vector_1 = vector_1 / np.linalg.norm(vector_1)
unit_vector_2 = vector_2 / np.linalg.norm(vector_2)
dot_product = np.dot(unit_vector_1, unit_vector_2)
angle = np.arccos(dot_product)

In [ ]:
unit_vector_1

In [ ]:
np.linalg.norm(vector_1)

In [ ]:
np.linalg.norm(vector_2)

In [ ]:
unit_vector_1

In [ ]:

p0=(0,1236.111911



)
p1=(1,1238.181779





)
p2=(2,1237.873559





)
p3=(30,1218.149039)

In [ ]:
v1=[p0[0]-p1[0],p0[1]-p1[1]]
v2=[p2[0]-p1[0],p2[1]-p1[1]]

In [ ]:
v1,v2

In [ ]:
def magnitude(v):
    return (v[0]**2+v[1]**2)**(1/2)
    

In [ ]:
np.dot(v1,v2)/(magnitude(v1)*magnitude(v2))

In [ ]:
np.arccos(np.dot(v1,v2)/(magnitude(v1)*magnitude(v2)))*180/3.14

In [ ]:
import logging
import os
import pandas as pd
from common import indicators
import sys
from datetime import datetime, timedelta
from common.get_api_data import *

logging.basicConfig(level=logging.INFO)
import common.application as ap
import pytz
#from simulator import place_order as simulator
from common.SlackUtil import sendMessage


In [ ]:
def getDateTime():
    datetime_obj_hour_fwd = datetime.now(tz)+timedelta(seconds=74)
    return datetime_obj_hour_fwd

from_date = str(getDateTime() - timedelta(days=14)).split(" ")[0]
app_properties = ap.app_properties
token_mappings = ap.token_mappings
csv_mapping = ap.csv_mapping

api_key = app_properties['api_key']
api_secret = app_properties['api_secret']
token ="340481"
os.makedirs(token, exist_ok=True)
file_name = "trades.csv"
tz = pytz.timezone('Asia/Kolkata')
logger = logging.getLogger('algo_tester')
# global define
profit = 0.019
stop_loss = 0.0095
historical_data = ""
rsi = 0.00
rsi_slope = 0.00
last_close = 0.00
last_high = 0.00
last_low = 0.00
last_price = get_price(token, 0.00)
current_price = 0.00
holding = ""
order_id = ""
swing = False
flag = ''
interval=10
rsi_count=0
sma_triggered=False
extrema_triggered=False
activation=''

In [ ]:
global historical_data, rsi, rsi_slope, last_close, swing,last_high,last_low,sma,ub,mb,lb,band_diff_trend,current_price
swing = False
to_date = str(getDateTime() - timedelta(minutes=2)).split(".")[0]
current_price = get_price(token, 0)
historical_data = get_data(token, from_date, to_date, "10minute", historical_data)
df = pd.DataFrame(historical_data)
df = indicators.RSI(df,period=14)
df=indicators.SMA(df, "close", "sma_7", 7)
df['RSI_Trend'] = df['RSI_14'].diff()
window = 20
no_of_std = 2
rolling_mean = df['close'].rolling(window).mean()
rolling_std = df['close'].rolling(window).std()
df['ub'] = rolling_mean + (rolling_std * no_of_std)
df['mb'] = rolling_mean
df['lb'] = rolling_mean - (rolling_std * no_of_std)
df['band_diff']=(df['ub']-df['lb']).round(2)
df['band_diff_trend'] = df['band_diff'].diff().round(0)
tail_dict = df.tail(1).to_dict('list')

sma = tail_dict['sma_7'][0]
rsi = tail_dict['RSI_14'][0]
rsi_slope = tail_dict['RSI_Trend'][0]
last_close = tail_dict['close'][0]
last_low = tail_dict['low'][0]
last_high = tail_dict['high'][0]
ub=tail_dict['ub'][0]
mb=tail_dict['mb'][0]
lb=tail_dict['lb'][0]
band_diff_trend=tail_dict['band_diff_trend'][0]

In [ ]:
df[-3:]

In [ ]:
ub_array=list(df.ub[-3:])

In [ ]:
def magnitude(v):
    return (v[0]**2+v[1]**2)**(1/2)

def compute_angle(arr):
    arr=arr.split(",")
    p0=(0,float(arr[0]))
    p1=(10,float(arr[1]))
    p2=(20,float(arr[2]))
    v1=[p0[0]-p1[0],p0[1]-p1[1]]
    v2=[p2[0]-p1[0],p2[1]-p1[1]]
    m1=magnitude(v1)
    m2=magnitude(v2)
    
    return 180-np.arccos(np.dot(v1,v2)/(magnitude(v1)*magnitude(v2)))*180/3.14



In [ ]:
global historical_data, rsi, rsi_slope, last_close, swing,last_high,last_low,sma,ub,mb,lb,band_diff_trend,current_price
swing = False
to_date = '2020-04-23 10:45:19'
current_price = get_price(token, 0)
historical_data = get_data(token, from_date, to_date, "10minute", historical_data)
df = pd.DataFrame(historical_data)
df = indicators.RSI(df,period=14)
df=indicators.SMA(df, "close", "sma_7", 7)
df['RSI_Trend'] = df['RSI_14'].diff()
window = 20
no_of_std = 2
rolling_mean = df['close'].rolling(window).mean()
rolling_std = df['close'].rolling(window).std()
df['ub'] = rolling_mean + (rolling_std * no_of_std)
df['mb'] = rolling_mean
df['lb'] = rolling_mean - (rolling_std * no_of_std)
df['band_diff']=(df['ub']-df['lb']).round(2)
df['band_diff_trend'] = df['band_diff'].diff().round(0)
df['3_ub_point']=df.ub.shift(2).apply(str)+","+    df.ub.shift(1).apply(str)+","+    df.ub.apply(str)
df['3_lb_point']=df.lb.shift(2).apply(str)+","+    df.lb.shift(1).apply(str)+","+    df.lb.apply(str)
df['divergence_angle']=df['3_ub_point'].apply(compute_angle)+df['3_lb_point'].apply(compute_angle)
df['divergence_col']=df.apply(lambda x:  "divergence" if 40 <=x['divergence_angle']<=80 and x['band_diff']>0 else '',axis=1)
df.to_csv("reliance.csv")

In [ ]:
df

In [ ]:
str(getDateTime() - timedelta(minutes=2)).split(".")[0]

In [ ]:
x=df
x['derived']=x.f_ub.apply(str)+","+x.s_ub.apply(str)+","+x.ub.apply(str)

In [ ]:
x['derived']

In [ ]:
global historical_data, rsi, rsi_slope, last_close, swing,last_high,last_low,sma,ub,mb,lb,band_diff_trend,current_price
swing = False
to_date = '2020-04-23 10:50:19'
current_price = get_price(token, 0)
historical_data = get_data(token, from_date, to_date, "10minute", historical_data)
df = pd.DataFrame(historical_data)
df[-1:]

In [ ]:
global historical_data, rsi, rsi_slope, last_close, swing,last_high,last_low,sma,ub,mb,lb,band_diff_trend,current_price
swing = False
to_date = '2020-04-23 10:45:19'
current_price = get_price(token, 0)
historical_data = get_data(token, from_date, to_date, "10minute", historical_data)
df = pd.DataFrame(historical_data)
df[-1:]

In [ ]:
df

In [ ]:
1366.90
2020-04-23 10:35:00+05:30
1371.00
1361.00
1370.60
913768

In [9]:
global historical_data, rsi, rsi_slope, last_close, swing,last_high,last_low,sma,ub,mb,lb,band_diff_trend,current_price
swing = False
to_date = str(getDateTime() - timedelta(minutes=2)).split(".")[0]
to_date="2020-04-24 12:55"
historical_data = get_data(token, from_date, to_date, "10minute", historical_data)
df = pd.DataFrame(historical_data)
df = indicators.RSI(df,period=14)
df=indicators.SMA(df, "close", "sma_7", 7)
df['RSI_Trend'] = df['RSI_14'].diff()
window = 20
no_of_std = 2
rolling_mean = df['close'].rolling(window).mean()
rolling_std = df['close'].rolling(window).std()
df['ub'] = rolling_mean + (rolling_std * no_of_std)
df['mb'] = rolling_mean
df['lb'] = rolling_mean - (rolling_std * no_of_std)
df['ub_tren']= df['ub'].diff().round(0)
df['lb_tren']= df['lb'].diff().round(0)
df['3_ub_point']=df.ub.shift(2).apply(str)+","+    df.ub.shift(1).apply(str)+","+    df.ub.apply(str)
df['3_lb_point']=df.lb.shift(2).apply(str)+","+    df.lb.shift(1).apply(str)+","+    df.lb.apply(str)
df['divergence_angle']=df['3_ub_point'].apply(compute_angle)+df['3_lb_point'].apply(compute_angle)
df['divergence_col']=df.apply(lambda x:  "divergence" if 25 <=x['divergence_angle']<=100 and x['ub_tren']>=0 and x['lb_tren']<=0 and x['ub_tren']!=x['lb_tren'] else '',axis=1)
tail_dict = df.tail(1).to_dict('list')

sma = tail_dict['sma_7'][0]
rsi = tail_dict['RSI_14'][0]
rsi_slope = tail_dict['RSI_Trend'][0]
last_close = tail_dict['close'][0]
last_low = tail_dict['low'][0]
last_high = tail_dict['high'][0]
ub=tail_dict['ub'][0]
mb=tail_dict['mb'][0]
lb=tail_dict['lb'][0]


df.to_csv("reliance.csv")

get_data exception  0
get_data exception  1
get_data exception  2
get_data exception  3
returning old history_df 


ValueError: DataFrame constructor not properly called!

In [ ]:
kite=get_session()

In [25]:
import requests
import json
data=requests.get("https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=NSE:INFY&interval=5min&apikey=TBWTGY6OPT64GV03&datatype=json").content
data=json.loads(data)

In [41]:
pd.DataFrame(data['Time Series (5min)']).transpose().reindex()

,1. open,2. high,3. low,4. close,5. volume
2020-04-24 05:55:00,660.2500,661.4500,659.7500,661.0000,93016
2020-04-24 05:50:00,656.9000,660.4000,656.4500,660.2500,169936
2020-04-24 05:45:00,656.3500,657.6000,655.6500,656.7000,202198
2020-04-24 05:40:00,658.2000,658.8000,655.5000,656.2500,164571
2020-04-24 05:35:00,657.9500,659.1500,657.0000,658.0500,117139
2020-04-24 05:30:00,661.9500,662.0000,657.1000,657.8000,144115
2020-04-24 05:25:00,661.7000,662.3500,661.0000,662.0000,57376
2020-04-24 05:20:00,662.5500,663.0000,661.4000,661.7000,64709
2020-04-24 05:15:00,662.3000,663.8500,662.0000,662.5000,69642
2020-04-24 05:10:00,662.3500,663.6000,660.3000,662.0000,121396


In [27]:
data


{'Meta Data': {'1. Information': 'Intraday (5min) open, high, low, close prices and volume',
  '2. Symbol': 'NSE:INFY',
  '3. Last Refreshed': '2020-04-24 05:55:00',
  '4. Interval': '5min',
  '5. Output Size': 'Compact',
  '6. Time Zone': 'US/Eastern'},
 'Time Series (5min)': {'2020-04-24 05:55:00': {'1. open': '660.2500',
   '2. high': '661.4500',
   '3. low': '659.7500',
   '4. close': '661.0000',
   '5. volume': '93016'},
  '2020-04-24 05:50:00': {'1. open': '656.9000',
   '2. high': '660.4000',
   '3. low': '656.4500',
   '4. close': '660.2500',
   '5. volume': '169936'},
  '2020-04-24 05:45:00': {'1. open': '656.3500',
   '2. high': '657.6000',
   '3. low': '655.6500',
   '4. close': '656.7000',
   '5. volume': '202198'},
  '2020-04-24 05:40:00': {'1. open': '658.2000',
   '2. high': '658.8000',
   '3. low': '655.5000',
   '4. close': '656.2500',
   '5. volume': '164571'},
  '2020-04-24 05:35:00': {'1. open': '657.9500',
   '2. high': '659.1500',
   '3. low': '657.0000',
   '4. c

In [ ]:
import requests
import json
data=requests.get("https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=NSE:INFY&interval=5min&apikey=TBWTGY6OPT64GV03&datatype=csv").content
data=json.loads(data)

In [47]:
pd.DataFrame(requests.get("https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=NSE:INFY&interval=5min&apikey=TBWTGY6OPT64GV03&datatype=csv").content)

ValueError: DataFrame constructor not properly called!

In [51]:
r

In [55]:
import io

In [81]:
r=requests.get("https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=NSE:RELIANCE&interval=15min&apikey=TBWTGY6OPT64GV03&datatype=csv")
c=pd.read_csv(io.StringIO(r.content.decode('utf-8')))
c['timestamp']=pd.to_datetime(c.timestamp)+pd.to_timedelta(570, unit='m')

In [84]:
k=c.sort_values(by='timestamp', ascending=True)

In [89]:
pd.DataFrame(k.to_dict())

,timestamp,open,high,low,close,volume
0,2020-04-24 15:15:00,1413.35,1429.95,1411.15,1425.70,1734518
1,2020-04-24 15:00:00,1423.05,1423.65,1406.50,1413.40,2806741
2,2020-04-24 14:45:00,1441.30,1445.00,1418.20,1423.70,2658882
3,2020-04-24 14:30:00,1439.24,1447.10,1436.59,1441.44,1567983
4,2020-04-24 14:15:00,1435.85,1442.55,1431.00,1438.65,1342449
5,2020-04-24 14:00:00,1451.25,1452.45,1423.70,1435.80,3328571
6,2020-04-24 13:45:00,1455.95,1463.00,1445.50,1451.05,1688876
7,2020-04-24 13:30:00,1456.32,1463.93,1451.93,1456.12,1606630
8,2020-04-24 13:15:00,1465.45,1469.55,1455.00,1456.55,1810437
9,2020-04-24 13:00:00,1467.56,1491.58,1462.98,1465.26,4144879


TypeError: 'Timedelta' object is not subscriptable

,timestamp,open,high,low,close,volume
99,2020-04-23 13:25:00,654.60,656.80,654.60,656.80,53924
98,2020-04-23 13:30:00,656.90,657.67,655.97,656.00,40908
97,2020-04-23 13:35:00,655.95,656.00,654.50,655.70,60691
96,2020-04-23 13:40:00,655.70,657.40,655.70,657.00,49834
95,2020-04-23 13:45:00,657.00,659.50,656.95,659.35,96744
94,2020-04-23 13:50:00,659.35,665.00,658.55,664.35,370110
93,2020-04-23 13:55:00,664.35,667.40,663.10,666.60,351854
92,2020-04-23 14:00:00,666.30,669.00,665.60,669.00,197273
91,2020-04-23 14:05:00,668.90,673.95,668.65,672.00,610952
90,2020-04-23 14:10:00,672.20,672.25,670.45,671.85,172266


In [91]:
pd.read_csv(io.StringIO(r.content.decode(requests.get("https://www.alphavantage.co/query?function=GLOBAL_QUOTE&symbol=IBM&apikey=demo&datatype=csv").content)))

TypeError: decode() argument 1 must be str, not bytes

In [94]:
io.StringIO(r.content.decode(requests.get("https://www.alphavantage.co/query?function=GLOBAL_QUOTE&symbol=IBM&apikey=demo&datatype=csv").content))

TypeError: decode() argument 1 must be str, not bytes

In [107]:
r=requests.get("https://www.alphavantage.co/query?function=GLOBAL_QUOTE&symbol=IBM&apikey=demo&datatype=csv")
c=float(pd.read_csv(io.StringIO(r.content.decode('utf-8')))['price'])

In [108]:
c

124.72

In [109]:
pd.read_csv(io.StringIO(r.content.decode('utf-8')))

,symbol,open,high,low,price,volume,latestDay,previousClose,change,changePercent
0,IBM,122.41,125.0,120.7634,124.72,4920523,2020-04-24,121.35,3.37,2.7771%
